In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import accuracy_score


import tensorflow as tf
from tensorflow import keras
import seaborn as sbn

In [2]:
# import data TODO: Change the file path to the dataset this question needed
df = pd.read_csv('..\\source_data\\crude_gas_p2_df.csv')
df

,week,crude_oil,cpi,us_pop,public_roads,us_urban_pop,oil_production,djus_auto_index,auto_sales,us_real_gdp,usd_mex_exrate,gasoline
0,2009/1/4,42.40,93.262178,306035.0,4059352.000,244607104.0,4935,83.77,10.535732,14476.2498,13.657700,1.684
1,2009/1/11,44.46,91.817169,306085.5,4059506.692,244658433.9,4917,83.06,8.347124,16075.1555,13.899500,1.784
2,2009/1/18,36.73,91.122475,306136.0,4059661.385,244709763.8,5052,74.02,8.574011,15137.1916,14.025100,1.847
3,2009/1/25,42.15,87.888142,306186.5,4059816.077,244761093.7,5045,64.56,8.961935,15155.3440,14.255700,1.838
4,2009/2/1,42.70,82.715265,306237.0,4059970.769,244812423.6,5235,62.59,9.348876,15538.1713,14.119200,1.892
...,...,...,...,...,...,...,...,...,...,...,...,...
569,2019/12/1,58.07,112.084710,329314.0,4190039.827,270476205.2,12900,310.32,17.941803,19477.3091,19.286699,2.575
570,2019/12/8,57.64,109.471205,329335.8,4189928.462,270513569.8,12800,311.68,14.872529,19750.1111,18.997499,2.561
571,2019/12/15,59.25,108.183411,329357.6,4189817.096,270550934.3,12800,318.41,18.616565,20106.3904,18.938499,2.536
572,2019/12/22,60.75,108.707988,329379.4,4189705.731,270588298.9,12900,334.78,16.962709,20396.2701,18.844700,2.532


# Creating independent and dependent variables 

In [3]:
# calculate independent variables
periods = (1,2,3,5,10)
for i in periods:
    df['of_chg_{}'.format(i)] = df['crude_oil'].pct_change(periods = i)
    
# calculate dependent variable
df['gas_chg'] = df['gasoline'].pct_change(periods=1)

# eliminate the empty rows
df = df[11:]
df

,week,crude_oil,cpi,us_pop,public_roads,us_urban_pop,oil_production,djus_auto_index,auto_sales,us_real_gdp,usd_mex_exrate,gasoline,of_chg_1,of_chg_2,of_chg_3,of_chg_5,of_chg_10,gas_chg
11,2009/3/22,49.49,93.593485,306562.2,4061053.615,245171732.9,5432,65.02,10.622196,15121.9490,14.338800,1.962,0.083881,0.146132,0.204136,0.339740,0.113135,0.027225
12,2009/3/29,52.99,105.508484,306603.6,4061208.308,245223062.8,5480,71.65,7.626950,15312.9473,13.541800,2.046,0.070721,0.160534,0.227189,0.426380,0.442690,0.042813
13,2009/4/5,50.34,97.283677,306645.0,4061363.000,245274392.8,5469,79.32,9.304512,15389.4132,13.130000,2.037,-0.050009,0.017175,0.102497,0.224818,0.194306,-0.004399
14,2009/4/12,50.46,79.216384,306699.5,4061517.692,245325722.7,5482,88.05,8.453785,14744.1587,13.097800,2.051,0.002384,-0.047745,0.019600,0.168597,0.181733,0.006873
15,2009/4/19,49.86,95.223458,306754.0,4061672.385,245377052.6,5421,89.79,9.155383,14624.3705,13.550500,2.059,-0.011891,-0.009535,-0.059068,0.091984,0.222658,0.003901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,2019/12/1,58.07,112.084710,329314.0,4190039.827,270476205.2,12900,310.32,17.941803,19477.3091,19.286699,2.575,0.020562,0.021460,0.024343,0.051993,-0.021237,-0.001551
570,2019/12/8,57.64,109.471205,329335.8,4189928.462,270513569.8,12800,311.68,14.872529,19750.1111,18.997499,2.561,-0.007405,0.013005,0.013896,0.044771,0.013005,-0.005437
571,2019/12/15,59.25,108.183411,329357.6,4189817.096,270550934.3,12800,318.41,18.616565,20106.3904,18.938499,2.536,0.027932,0.020320,0.041301,0.045158,0.115399,-0.009762
572,2019/12/22,60.75,108.707988,329379.4,4189705.731,270588298.9,12900,334.78,16.962709,20396.2701,18.844700,2.532,0.025316,0.053956,0.046151,0.068602,0.140417,-0.001577


# Seperating training, validation, and test datasets

In [4]:
# create list of independent variable's name TODO: add the appropriate variable names to the list
ind_var_name = ['of_chg_{}'.format(i) for i in periods]+['cpi','us_pop','public_roads',
                                                         'us_urban_pop','oil_production','djus_auto_index',
                                                         'auto_sales','us_real_gdp','usd_mex_exrate']
num_var =len(ind_var_name) # put number of variables here

# extract the values to X
X = df[['crude_oil'] + ind_var_name].values
X

array([[ 4.94900000e+01,  8.38808585e-02,  1.46132469e-01, ...,
         1.06221963e+01,  1.51219490e+04,  1.43388004e+01],
       [ 5.29900000e+01,  7.07213579e-02,  1.60534385e-01, ...,
         7.62694980e+00,  1.53129473e+04,  1.35417996e+01],
       [ 5.03400000e+01, -5.00094357e-02,  1.71751869e-02, ...,
         9.30451160e+00,  1.53894132e+04,  1.31300001e+01],
       ...,
       [ 5.92500000e+01,  2.79319917e-02,  2.03203031e-02, ...,
         1.86165647e+01,  2.01063904e+04,  1.89384994e+01],
       [ 6.07500000e+01,  2.53164557e-02,  5.39555864e-02, ...,
         1.69627090e+01,  2.03962701e+04,  1.88446999e+01],
       [ 6.12900000e+01,  8.88888889e-03,  3.44303797e-02, ...,
         1.78387504e+01,  1.88128854e+04,  1.89113007e+01]])

In [5]:
# create y for regressor, y_c for classifier
y = df['gas_chg']
y_c = (df['gas_chg'] > 0).values.astype('int')

# set training, validation, and test criterias
training_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

# check correctness of X and y
X.shape, y.shape, y_c.shape

((563, 15), (563,), (563,))

In [6]:
# create training, validation, and test sets for regressor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_ratio, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = validation_ratio/(validation_ratio + training_ratio), random_state = 42)
print("X_train shape:\t", X_train.shape)
print("X_test shape:\t", X_test.shape)
print("X_val shape:\t", X_val.shape)
print("y_train shape:\t", y_train.shape)
print("y_val shape:\t", y_val.shape)
print("y_test shape:\t", y_test.shape)

X_train shape:	 (393, 15)
X_test shape:	 (85, 15)
X_val shape:	 (85, 15)
y_train shape:	 (393,)
y_val shape:	 (85,)
y_test shape:	 (85,)


In [7]:
# create training, validation, and test sets for classifier
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_c, test_size = test_ratio, random_state = 42)
X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(X_train_c, y_train_c, test_size = validation_ratio/(validation_ratio + training_ratio), random_state = 42)
print("X_train_c shape:", X_train_c.shape)
print("X_test_c shape:\t", X_test_c.shape)
print("X_val_c shape:\t", X_val_c.shape)
print("y_train_c shape:", y_train_c.shape)
print("y_val_c shape:\t", y_val_c.shape)
print("y_test_c shape:\t", y_test_c.shape)

X_train_c shape: (393, 15)
X_test_c shape:	 (85, 15)
X_val_c shape:	 (85, 15)
y_train_c shape: (393,)
y_val_c shape:	 (85,)
y_test_c shape:	 (85,)


# Principal Component Analysis (PCA)

In [8]:
# normalization for regressor set
for i in range(X_train.shape[1]):
    X_train_mean = X_train[:, i]. mean()
    X_train_std = X_train[:, i].std()
    X_train[:, i] = (X_train[:,i] - X_train_mean) / X_train_std
    X_test[:,i] = (X_test[:,i] - X_train_mean) / X_train_std
    X_val[:, i] = (X_val[:,i] - X_train_mean) / X_train_std

print("X_train shape:\t", X_train.shape)
print("X_val shape:\t", X_val.shape)

X_train shape:	 (393, 15)
X_val shape:	 (85, 15)


In [9]:
# normalization for classifier set
for i in range(X_train_c.shape[1]):
    X_train_c_mean = X_train_c[:, i]. mean()
    X_train_c_std = X_train_c[:, i].std()
    X_train_c[:, i] = (X_train_c[:,i] - X_train_c_mean) / X_train_c_std
    X_test_c[:,i] = (X_test_c[:,i] - X_train_c_mean) / X_train_c_std
    X_val_c[:, i] = (X_val_c[:,i] - X_train_c_mean) / X_train_c_std

print("X_train_c shape:", X_train_c.shape)
print("X_val_c shape:\t", X_val_c.shape)

X_train_c shape: (393, 15)
X_val_c shape:	 (85, 15)


In [10]:
# PCA for regressor
# fit on training set
scaler = StandardScaler()
scaler.fit(X_train)

# apply transform to training and test set
X_train_pca = scaler.transform(X_train)
X_test_pca = scaler.transform(X_test)
X_val_pca = scaler.transform(X_val)

# set PCA value
pca = PCA(0.95)

# fit PCA training set
pca.fit(X_train_pca)

# apply transform to training and test set
X_train_pca = pca.transform(X_train_pca)
X_test_pca = pca.transform(X_test_pca)
X_val_pca = pca.transform(X_val_pca)
print("X_train_pca shape:\t", X_train_pca.shape)
print("X_test_pca shape:\t", X_test_pca.shape)
print("X_val_pca shape:\t", X_val_pca.shape)
print("y_train shape:\t", y_train.shape)
print("y_val shape:\t", y_val.shape)
print("y_test shape:\t", y_test.shape)

X_train_pca shape:	 (393, 8)
X_test_pca shape:	 (85, 8)
X_val_pca shape:	 (85, 8)
y_train shape:	 (393,)
y_val shape:	 (85,)
y_test shape:	 (85,)


In [11]:
# PCA for classifier
# fit on training set_c
scaler = StandardScaler()
scaler.fit(X_train_c)

# apply transform to training and test set
X_train_c_pca = scaler.transform(X_train_c)
X_test_c_pca = scaler.transform(X_test_c)
X_val_c_pca = scaler.transform(X_val_c)

# set PCA value
pca = PCA(0.95)

# fit PCA training set
pca.fit(X_train_c_pca)

# apply transform to training and test set
X_train_c_pca = pca.transform(X_train_c_pca)
X_test_c_pca = pca.transform(X_test_c_pca)
X_val_c_pca = pca.transform(X_val_c_pca)
print("X_train_c_pca shape:\t", X_train_c_pca.shape)
print("X_test_c_pca shape:\t", X_test_c_pca.shape)
print("X_val_c_pca shape:\t", X_val_c_pca.shape)
print("y_train_c shape:", y_train_c.shape)
print("y_val_c shape:\t", y_val_c.shape)
print("y_test_c shape:\t", y_test_c.shape)

X_train_c_pca shape:	 (393, 8)
X_test_c_pca shape:	 (85, 8)
X_val_c_pca shape:	 (85, 8)
y_train_c shape: (393,)
y_val_c shape:	 (85,)
y_test_c shape:	 (85,)


In [12]:
num_var = X_train_c_pca.shape[1]
num_var

8

# Multiple linear regression

In [13]:
multiple_reg = LinearRegression().fit(X_train_pca, y_train)
score_multiple_reg = multiple_reg.score(X_test_pca, y_test)
print('score_multiple_reg:', "%.4f"%score_multiple_reg)

score_multiple_reg: 0.4741


# Multiple polynomial regression

In [14]:
#to prove this is the correct polynomial regression, set degree to 1
poly = PolynomialFeatures(1)
# polynomial transform
X_train_poly = poly.fit_transform(X_train_pca, y_train)
X_val_poly = poly.fit_transform(X_val_pca,y_val)
X_test_poly = poly.fit_transform(X_test_pca,y_test)

# fit to linear regression
polyreg = LinearRegression().fit(X_train_poly, y_train)

# predict using liear regression
score_poly_reg = polyreg.score(X_test_poly, y_test)
score_poly_reg

0.47410825221635733

In [15]:
# use val to find the appropriate degree
mpr = dict()
for i in range(2,6):
    degree=i
    poly = PolynomialFeatures(degree)
    # polynomial transform
    X_train_poly = poly.fit_transform(X_train_pca, y_train)
    X_val_poly = poly.fit_transform(X_val_pca,y_val)
    X_test_poly = poly.fit_transform(X_test_pca,y_test)

    # fit to linear regression
    polyreg = LinearRegression().fit(X_train_poly, y_train)

    # predict using liear regression
    score_poly_reg = polyreg.score(X_val_poly, y_val)
    print('score_poly_reg, degree',i,':', "%.4f"%score_poly_reg)
    mpr[f'{i}'] = score_poly_reg
    
max_degree = max(mpr, key = mpr.get)
print('max degree:', max_degree)

score_poly_reg, degree 2 : 0.4858
score_poly_reg, degree 3 : -0.8378
score_poly_reg, degree 4 : -183.8014
score_poly_reg, degree 5 : -52.2767
max degree: 2


In [16]:
poly = PolynomialFeatures(int(max_degree))
# polynomial transform
X_train_poly = poly.fit_transform(X_train_pca, y_train)
X_val_poly = poly.fit_transform(X_val_pca,y_val)
X_test_poly = poly.fit_transform(X_test_pca,y_test)

# fit to linear regression
polyreg = LinearRegression().fit(X_train_poly, y_train)

# predict using liear regression
score_poly_reg = polyreg.score(X_test_poly, y_test)
score_poly_reg

0.411518953833505

# Random forest classifier

In [17]:
# testing for different max depth
rfc = dict()
for i in range (1, 15):
    clf = RandomForestClassifier(max_depth=i, random_state=0).fit(X_train_c_pca, y_train_c)
    y_pred = clf.predict(X_val_c_pca)
    score_clf = accuracy_score(y_val_c, y_pred)
    rfc[f'{i}'] = score_clf
    #print('score_clf_',i,":", "%.4f"%score_clf)
max_depth = max(rfc, key=rfc.get)
print('max depth:', max_depth)

max depth: 5


In [18]:
# choose max_depth = 4
clf = RandomForestClassifier(max_depth=int(max_depth), random_state=0).fit(X_train_c_pca, y_train_c)
score_rf_clf = clf.score(X_test_c_pca, y_test_c)
print('score_rf_clf:', "%.4f"%score_rf_clf)

score_rf_clf: 0.7765


# Random forest regressor

In [19]:
clf_reg = RandomForestRegressor().fit(X_train_pca, y_train)
score_rf_reg = clf_reg.score(X_test_pca, y_test)
print('score_rf_reg:', "%.4f"%score_rf_reg)

score_rf_reg: 0.4757


# Logistic regression

In [20]:
logistic_model = LogisticRegression(penalty = 'l1', solver='liblinear', random_state=0).fit(X_train_c_pca, y_train_c)
score_logistic = logistic_model.score(X_test_c_pca, y_test_c)
print('score_logistic:', "%.4f"%score_logistic)

score_logistic: 0.7765


# Gradient boosting classifier

In [21]:
gra_clf = GradientBoostingClassifier(n_estimators = 200, learning_rate=0.50, max_depth=5, random_state=0).fit(X_train_c_pca, y_train_c)
score_gb_clf = gra_clf.score(X_test_c_pca, y_test_c)
score_gb_clf

0.7411764705882353

# Gradient boosting regressor 

In [22]:
gra_reg = GradientBoostingRegressor(random_state=0).fit(X_train_pca, y_train)
score_gb_reg = gra_reg.score(X_test_pca, y_test)
score_gb_reg

0.4714580445818649

# XGBoost classifier

In [23]:
xgb_clf = XGBClassifier().fit(X_train_c_pca, y_train_c)
score_xgb_clf = xgb_clf.score(X_test_c_pca, y_test_c)
score_xgb_clf

[16:19:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\lemon\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.788235294117647

# XGBoost regressor

In [24]:
xgb_reg = XGBRegressor(random_state = 0).fit(X_train_pca, y_train)
score_xgb_reg = xgb_reg.score(X_test_pca, y_test)
score_xgb_reg

0.4175684203606762

# AdaBoost classifier

In [25]:
adb_clf = AdaBoostClassifier().fit(X_train_c_pca, y_train_c)
score_adb_clf = adb_clf.score(X_test_c_pca, y_test_c)
score_adb_clf

0.7411764705882353

# AdaBoost regressor

In [26]:
adb_reg = AdaBoostRegressor().fit(X_train_pca, y_train)
score_adb_reg = adb_reg.score(X_test_pca, y_test)
score_adb_reg

0.41085684356322394

# Support vector classification

In [27]:
# Try different kernels
kernels = ('linear', 'poly', 'rbf')

for k in kernels:
    svm_clf = SVC(kernel=k).fit(X_train_c_pca, y_train_c)
    y_pred = svm_clf.predict(X_test_c_pca)
    vars()[f'score_svc_{k}']= accuracy_score(y_test_c, y_pred)
    
print(score_svc_linear)
print(score_svc_poly)
print(score_svc_rbf)

0.7411764705882353
0.7411764705882353
0.7529411764705882


# Gaussian Naive Bayes

In [28]:
gnb_clf = GaussianNB().fit(X_train_c_pca, y_train_c)
score_gnb = gnb_clf.score(X_test_c_pca, y_test_c)
print(score_gnb)

0.7529411764705882


# Neuro networks

In [29]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(10, input_dim=num_var, activation='relu'))
model.add(keras.layers.Dense(5, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_c_pca, y_train_c, epochs=50, validation_data=(X_val_c_pca, y_val_c))

Epoch 1/50
13/13 [==============================] - 1s 78ms/step - loss: 0.6659 - accuracy: 0.5814 - val_loss: 0.7115 - val_accuracy: 0.5176
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 0.6353 - accuracy: 0.5868 - val_loss: 0.6941 - val_accuracy: 0.5294
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 0.6160 - accuracy: 0.6237 - val_loss: 0.6797 - val_accuracy: 0.5529
Epoch 4/50
13/13 [==============================] - 0s 3ms/step - loss: 0.6025 - accuracy: 0.6534 - val_loss: 0.6662 - val_accuracy: 0.5529
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 0.6220 - accuracy: 0.6475 - val_loss: 0.6539 - val_accuracy: 0.5882
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.5909 - accuracy: 0.7022 - val_loss: 0.6446 - val_accuracy: 0.5882
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 0.5688 - accuracy: 0.7290 - val_loss: 0.6347 - val_accuracy: 0.6353
Epoch 8/50
13/13 [=

In [30]:
loss, score_neuro = model.evaluate(X_test_c_pca, y_test_c)
print(score_neuro)

3/3 [==============================] - 0s 998us/step - loss: 0.4657 - accuracy: 0.8000
0.800000011920929


# Summary

In [31]:
reg = {
    'Multiple linear regression': score_multiple_reg,
    'Multiple polynomial regression': score_poly_reg,
    'Random forest regressor': score_rf_reg,
    'Gradient boosting regressor': score_gb_reg,
    'XGBoost regressor': score_xgb_reg,
    'AdaBoost regressor': score_adb_reg
}
reg_df = pd.DataFrame.from_dict(reg, orient='index', columns = ['Score'])
reg_df.index.name = 'Regression models'
#reg_df.sort_values(by=['Score'], ascending=False)
reg_df

,Score
Regression models,
Multiple linear regression,0.474108
Multiple polynomial regression,0.411519
Random forest regressor,0.475670
Gradient boosting regressor,0.471458
XGBoost regressor,0.417568
AdaBoost regressor,0.410857


In [32]:
clf = {
    'Logistic regression': score_logistic,
    'Random forest classifier': score_rf_clf,
    'Gradient boosting classifier': score_gb_clf,
    'XGBoost classifier': score_xgb_clf,
    'AdaBoost classifier': score_adb_clf,
    'Gaussian naive Bayes': score_gnb, 
    'SVC linear': score_svc_linear,
    'SVC polynomial': score_svc_poly, 
    'SVC RBF': score_svc_rbf,
    'Neuro network' : score_neuro
}
clf_df = pd.DataFrame.from_dict(clf, orient='index',columns = ['Score'])
clf_df.index.name = 'Classifier models'
#clf_df.sort_values(by=['Score'], ascending=False)
clf_df

,Score
Classifier models,
Logistic regression,0.776471
Random forest classifier,0.776471
Gradient boosting classifier,0.741176
XGBoost classifier,0.788235
AdaBoost classifier,0.741176
Gaussian naive Bayes,0.752941
SVC linear,0.741176
SVC polynomial,0.741176
SVC RBF,0.752941
